In [1]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os, json
import kfp
import kfp.compiler as compiler

In [3]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)

# Input and output parameters for the pipeline run

In [4]:
project_id = "ynli7c"
image = "docker.io/ocdr/dkube-datascience-tf-cpu:fs-v2.0.0"
ptrain_dataset = 'titanic-train'
ptest_dataset = 'titanic-test'
dataset_mount_points = ["/opt/dkube/input/train", "/opt/dkube/input/test"]
train_fs = 'titanic-train-fs'
test_fs = 'titanic-test-fs'
output_featureset_mount_points = ["/opt/dkube/output/train", "/opt/dkube/output/test"]
training_program = 'titanic'
preprocessing_script = f"python preprocessing.py --train_fs {train_fs} --test_fs {test_fs}"
training_script = "python training.py"
train_inp_mount_points = ["/titanic-train","/titanic-test"]
train_out_mount_points = ["/model"]
model = 'titanic'
user = os.getenv('USERNAME')

# Loading components

In [5]:
components_url = "https://raw.githubusercontent.com/oneconvergence/dkube/master/components/"
dkube_preprocessing_op = kfp.components.load_component_from_url(components_url + "preprocess/component.yaml")
dkube_training_op = kfp.components.load_component_from_url(components_url + "training/component.yaml")
runid = 0

# Pipeline Definition

In [6]:
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='example titanic pipeline to submit to leaderboard'
)
def titanic_pipeline(token, project_id):

    preprocessing = dkube_preprocessing_op(token, json.dumps({"image": image}),
                                           program=training_program, run_script=preprocessing_script,
                                           datasets=json.dumps([ptrain_dataset, ptest_dataset]), 
                                           output_featuresets=json.dumps([train_fs, test_fs]),
                                           input_dataset_mounts=json.dumps(dataset_mount_points), 
                                           output_featureset_mounts=json.dumps(output_featureset_mount_points)
                                            )

    train       = dkube_training_op(token, json.dumps({"image": image}),
                                framework="sklearn", version="0.23.2",
                                program=training_program, run_script=training_script,
                                featuresets=json.dumps([train_fs, test_fs]), outputs=json.dumps([model]),
                                input_featureset_mounts=json.dumps(train_inp_mount_points),
                                output_mounts=json.dumps(train_out_mount_points)).after(preprocessing)

# Compling pipeline into tar file

In [7]:
pl_file_name = 'titanic_pre_train_pl.tar.gz'
pl_name = 'titanic_pre_train'
compiler.Compiler().compile(titanic_pipeline, pl_file_name)
# Upload this generated tarball into the Pipelines UI

# Uploading Pipeline

In [8]:
client = kfp.Client(existing_token=existing_token)
try:
  client.upload_pipeline(pipeline_package_path = pl_file_name, pipeline_name = pl_name, description = None)
except BaseException as e:
    print(e)

# Creating experiment

In [9]:
try:
    titanic_experiment = client.create_experiment(name='Dkube - Titanic')
except BaseException as e:
    print(e)

# Creating run from pipeline under the titanic experiment

In [10]:
try:
    pipeline_id = client.get_pipeline_id(pl_name)
    run = client.run_pipeline(titanic_experiment.id, 'titanic_fs_pl', None, pipeline_id=pipeline_id,
                              params={"token":existing_token, "project_id":project_id})
except BaseException as e:
    print(e)

In [ ]:
# client = kfp.Client(existing_token=existing_token)
# #Replace [titanic] & wprz8s with your project name and id respectively.
# client.create_run_from_pipeline_func(titanic_pipeline, run_name="[titanic] Run" + str(runid), arguments={"token":existing_token,"project_id":"ynli7c"}, experiment_name = titanic_experiment.name)
# runid += 1